In [ ]:
# more optimized
import numpy as np

class Conv2D:
  """
  Simple implementation of a 2D convolutional layer.
  """

  def __init__(self, input_channels, output_channels, kernel_size, padding='valid', activation='relu'):
    """
    Initialize the convolutional layer.

    Args:
      input_channels: Number of channels in the input data.
      output_channels: Number of filters (feature maps) to learn.
      kernel_size: Size of the convolutional kernel (filter).
      padding: Padding mode ('valid' or 'same').
      activation: Activation function to apply (e.g., 'relu', 'sigmoid').
    """

    self.input_channels = input_channels
    self.output_channels = output_channels
    self.kernel_size = kernel_size
    self.padding = padding
    self.activation = activation

    # Initialize weights and biases with random values
    self.weights = np.random.randn( output_channels,input_channels, kernel_size, kernel_size) / np.sqrt(input_channels * kernel_size * kernel_size)
    #self.weights = self.weights.reshape((kernel_size,kernel_size,input_channels))
    self.bias = np.zeros((output_channels,))
    #print(f"====weight \n {self.weights}  \n\n ====Type \n {self.weights.dtype}\n\n =====shape\n {self.weights.shape}")


  def forward(self, X):
    """
    Perform the forward pass of the convolution operation.

    Args:
      X: Input data (tensor of shape (batch_size, input_height, input_width, input_channels)).

    Returns:
      Output of the convolutional layer (tensor of appropriate shape based on padding).
    """

    batch_size, input_height, input_width, _ = X.shape

    # Handle padding
    if self.padding == 'same':
      pad_amount = (self.kernel_size - 1) // 2
      X_padded = np.pad(X, ((0,0),(pad_amount, pad_amount), (pad_amount, pad_amount), (0, 0)), mode='constant')
    else:
      X_padded = X

    output_height = input_height - self.kernel_size + 1 if self.padding == 'valid' else input_height
    output_width = input_width - self.kernel_size + 1 if self.padding == 'valid' else input_width

    # Perform convolution for each output channel
    output = np.zeros((batch_size, output_height, output_width, self.output_channels))
    for n in range(batch_size):
      for out_channel in range(self.output_channels):
        for i in range(output_height):
          for j in range(output_width):
            region=X_padded[n,i:i+self.kernel_size,j:j+self.kernel_size,:]
            output[n, i, j, out_channel] = np.sum(region * self.weights[out_channel,:,:,:].transpose(1,2,0)) +self.bias[out_channel]

    # Apply activation function
    if self.activation == 'relu':
      output = np.maximum(output, 0)
    elif self.activation == 'sigmoid':
      output = 1 / (1 + np.exp(-output))
    else:
      raise NotImplementedError(f"Activation func '{self.activation}' not supported")

    return output



class MaxPooling:
  """
  Simple implementation of a max pooling layer.
  """

  def __init__(self, pool_size):
    """
    Initialize the max pooling layer.

    Args:
      pool_size: Size of the pooling window.
    """

    self.pool_size = pool_size

  def forward(self, X):
    """
    Perform the forward pass of the max pooling operation.

    Args:
      X: Input data (tensor of shape (batch_size, input_height, input_width, input_channels)).

    Returns:
      Output of the max pooling layer (tensor of appropriate shape).
    """

    batch_size, input_height, input_width, input_channels = X.shape
    output_height = input_height // self.pool_size
    output_width = input_width // self.pool_size

    output = np.zeros((batch_size, output_height, output_width, input_channels))
    for n in range(batch_size):
      for i in range(output_height):
        for j in range(output_width):
          # Extract region
          region = X[n, i * self.pool_size:(i +1) * self.pool_size, j * self.pool_size:(j+1) * self.pool_size,:]
          # Find maximum value and store
          output[n,i,j,:]=np.max(region,axis=(0,1))

    return output



class SimpleCNN:
  def __init__(self):
    # Define layer parameters (replace with desired values)
    self.conv1 = Conv2D(3, 16, 3, padding='same', activation='relu')
    self.pool1 = MaxPooling(2)
    self.conv2 = Conv2D(16, 32, 3, padding='tanh', activation='relu')
    self.pool2 = MaxPooling(2)
    self.conv3 = Conv2D(32, 64, 3, padding='valid', activation='relu')

  def forward(self, X):
    # Pass data through each layer sequentially
    X = self.conv1.forward(X)
    X = self.pool1.forward(X)
    X = self.conv2.forward(X)
    X = self.pool2.forward(X)
    X = self.conv3.forward(X)
    return X

# Example usage
model = SimpleCNN()
batch_size = 5
input_data = np.random.randn(batch_size, 20, 20, 3)  # Replace with your actual data
output = model.forward(input_data)

print(output.shape)  # Output shape will depend on your chosen pool sizes
output.item,output.size,output.ndim